In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

URL = 'https://www.audit-it.ru/archive/'
rg = 'https://www.audit-it.ru'
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

results = soup.find_all(class_="archive_year")
news = []

for result in results:
    data = result.find(class_="archive_block_title")
    if (data.getText() in ["2022"]):
        for newsFound in result.find_all('a', href=True):
            news.append(newsFound['href'])

sites = []
for newsSite in news:
    page_news = requests.get(rg + newsSite)
    news_soup = BeautifulSoup(page_news.content, "html.parser")
    for textNews in news_soup.find_all(class_="archive_line"):
        for site in textNews.find_all('a', href=True):
            sites.append(site['href'])

listOfNews = []
for newsSite in sites:
    page_news = requests.get(rg + newsSite, headers = {'User-agent': 'Super Bot Power Level Over 9000'})
    
    news_soup = BeautifulSoup(page_news.content, "html.parser")
    for textNews in news_soup.find_all(class_="news-title-box"):
        for site in textNews.find_all('a', href=True):
            listOfNews.append(site['href'])

list_of_lists = []
for newsSite in listOfNews:
    page_news = requests.get(rg + newsSite, headers = {'User-agent': 'Super Bot Power Level Over 9000'})
    news_soup = BeautifulSoup(page_news.content, "html.parser")
    if ('articles' in newsSite):
        textPage = news_soup.find(class_ = 'article-text')
    elif ('law'in newsSite):
        textPage = news_soup.find(class_ = 'article-text')
    else:
        textPage = news_soup.find(class_ = 'news-text')
    a = listOfNews.index(newsSite)
    s = rg + newsSite
    maintitle = news_soup.find(class_ = 'maintitle')
    if maintitle: maintitle = maintitle.text
    datePublished = news_soup.find(itemprop='datePublished').text
    
    alltext = []
    for text in textPage.find_all('p'):
        alltext.append(text.text)
    list_of_lists.append([maintitle, datePublished, alltext, rg + newsSite])
df = pd.DataFrame(list_of_lists, columns=['Title', 'Date', 'Text', 'Site'])

df.to_csv (r'/content/export_dataframe.csv', index = False, header=True, encoding='utf-8')